In [2]:
import numpy as np
import os
import pickle

from tqdm import tqdm_notebook
import pandas as pd


In [43]:
mutation_labels = ["Y29F", "Y29A",  "W38F", "T42A", "W44F", "E76A", "E80A", 
                   "D35A", "D39A", "K27A", "R59A", "R83Q", "R87A", "H102A", 
                   "F29Y", "A29Y",  "F38W", "A42T", "F44W", "A76E", "A80E", 
                   "A35D", "A39D", "A27K", "A59R", "Q83R", "A87R", "A102H"]
categories = ["backbone torsions", "sidechain torsions", "intra interface contacts", "inter interface contacts", "neighboring waters"]
phase = 'complex'

data = []
for sub_dir in tqdm_notebook(range(28)):
    print(f"sub_dir: {sub_dir}")
    
    if sub_dir == 4:
        main_dir = 45
        sub_dir_final = 11
        replicate = 0

    elif sub_dir == 12:
        main_dir = 45
        sub_dir_final = 10
        replicate = 1

    elif sub_dir == 17:
        main_dir = 45
        sub_dir_final = 9
        replicate = 1
    
    else:
        main_dir = 47
        replicate = 1
        sub_dir_final = sub_dir

    # Load correlation data -- dict of key: name of degree of freedom, value: dict containing 'pearsonr' and 'tranform_type' as keys
    out_dir = f"/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/{main_dir}/{sub_dir_final}/replicate_{replicate}/"
    with open(os.path.join(out_dir, f"correlation_data_replicas_all_50ns.pickle"), "rb") as f:
        correlation_data = pickle.load(f)
    correlation_data_keys = list(correlation_data.keys())
    print(correlation_data.keys())
    
    # Load pccs and cis 
    with open(os.path.join(out_dir, f"correlation_data_replicas_all_with_cis_50ns.pickle"), "rb") as f:
        correlation_data_cis = pickle.load(f) # First 5 elements are PCCs for each of the dof categories, latter 5 elements are the corresponding CIs
    print(correlation_data_cis)
    
    for i, category in enumerate(categories):
        if i == 4:
            dof_name = f"nearby waters {correlation_data_keys[i]}"
        else:
            dof_name = correlation_data_keys[i]
        data.append([mutation_labels[sub_dir], category, round(correlation_data_cis[i], 2), tuple(round(val, 2) for val in correlation_data_cis[i+5]), dof_name])

/scratch/lsftmp/18362115.tmpdir/ipykernel_69306/554619522.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sub_dir in tqdm_notebook(range(28)):


  0%|          | 0/28 [00:00<?, ?it/s]

sub_dir: 0
odict_keys(['barstar-42 psi old', 'barstar-29 chi1 old', 'barstar-29 | barstar-43 old', 'barnase-87 | barstar-29 old', 'old'])
[0.028328824305813915, -0.21438671822060873, 0.16207117923213382, 0.1800885139786311, 0.18994794980115035, (-0.003360356930008182, 0.04949548499381088), (-0.25167355480676523, -0.17971739014664664), (0.12624577750679253, 0.1949840324291723), (0.14632459574481915, 0.20991953048040862), (0.15136438361192456, 0.22279847773336423)]
sub_dir: 1
odict_keys(['barstar-29 psi old', 'barstar-29 chi1 old', 'barstar-27 | barstar-29 old', 'barnase-102 | barstar-29 old', 'old'])
[0.05967799407330672, -0.1538170180698482, 0.11679754619061734, 0.16513722836851008, 0.2065381607083448, (0.033057021772187625, 0.07873237091250022), (-0.21051977250198683, -0.09089642551144247), (0.0713643294134853, 0.1555525284295122), (0.14348478774110632, 0.188737274335835), (0.1592621388652001, 0.24042007385695424)]
sub_dir: 2
odict_keys(['barnase-60 phi old', 'barstar-38 chi1 old', 'b

In [44]:
df = pd.DataFrame(data, columns=["Mutation", "Degree of freedom category", "PCC", "95% confidence interval", "Degree of freedom"])

In [45]:
df

,Mutation,Degree of freedom category,PCC,95% confidence interval,Degree of freedom
0,Y29F,backbone torsions,0.03,"(-0.0, 0.05)",barstar-42 psi old
1,Y29F,sidechain torsions,-0.21,"(-0.25, -0.18)",barstar-29 chi1 old
2,Y29F,intra interface contacts,0.16,"(0.13, 0.19)",barstar-29 | barstar-43 old
3,Y29F,inter interface contacts,0.18,"(0.15, 0.21)",barnase-87 | barstar-29 old
4,Y29F,neighboring waters,0.19,"(0.15, 0.22)",nearby waters old
...,...,...,...,...,...
135,A102H,backbone torsions,-0.08,"(-0.12, -0.04)",barstar-31 phi old
136,A102H,sidechain torsions,0.08,"(0.05, 0.11)",barnase-102 chi2 new
137,A102H,intra interface contacts,-0.14,"(-0.19, -0.1)",barstar-29 | barstar-35 old
138,A102H,inter interface contacts,-0.13,"(-0.18, -0.09)",barnase-58 | barstar-29 old


In [46]:
save_path = "/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/47/table_50ns_arex_pccs.csv"
df.to_csv(save_path, index=False)